In [4]:
import os
import opensmile
import pandas as pd
import numpy as np
from natsort import natsorted
import pandas as pd
from pydub import AudioSegment as AS
from ChaFile import *
from difflib import SequenceMatcher as sm
import editdistance as ed
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [5]:
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.Functionals,
)

dir_healthy_wav = input("enter the (full) path to your directory of healthy WAV files --> ")
dir_healthy_cha = input("enter the (full) path to your directory of healthy CHA files --> ")

dir_AD_wav = input("enter the (full) path to your directory of AD WAV files --> ")
dir_AD_cha = input("enter the (full) path to your directory of AD CHA files --> ")

dir_MCI_wav = input("enter the (full) path to your directory of MCI WAV files --> ")
dir_MCI_cha = input("enter the (full) path to your directory of MCI CHA files --> ")


In [6]:
def checker(wav, cha):
    files_from_dir = os.listdir(wav)
    files_from_dir_2 = os.listdir(cha)
    if len(files_from_dir) != len(files_from_dir_2):
        quit("not equal")

checker(dir_healthy_wav, dir_healthy_cha)
checker(dir_AD_wav, dir_AD_cha)
checker(dir_MCI_wav, dir_MCI_cha)

AD_cha_files = natsorted(os.listdir(dir_AD_cha))
AD_wav_files = natsorted(os.listdir(dir_AD_wav))

MCI_cha_files = natsorted(os.listdir(dir_MCI_cha))
MCI_wav_files = natsorted(os.listdir(dir_MCI_wav))

Healthy_cha_files = natsorted(os.listdir(dir_healthy_cha))
Healthy_wav_files = natsorted(os.listdir(dir_healthy_wav))

In [23]:
keys_1 = ["command_1", "command_2", "command_3", "command_4", "command_5", "command_6", "command_7", "command_8", "command_9", 
"command_10", "command_11", "command_12", "command_13", "command_14", "command_15", "command_16", "command_17", "command_18", "command_19",
"command_20", "command_21", "command_22", "command_23", "command_24", "command_25", "command_26", "command_27", "command_28", "command_29",
"command_30", "command_31", "command_32", "command_33"]

COMMANDS = ["Alexa what is the weather outside ? ",
"Alexa what is today's date ? ", 
"Alexa what time is it ? ", 
"Alexa when is Thanksgiving ? ",
"Alexa how do you bake chocolate chip cookies ? ",
"Alexa what is two times four ? ",
"Alexa how many tablespoons in a cup ? ",
"Alexa how do you spell symptom ? ",
"Alexa play classical music . ",
"Alexa volume eight . ", 
"Alexa play jazz . ",
"Alexa volume six . ",
"Alexa stop . ",
"Alexa remind me to start the laundry tomorrow at two p_m . ",
"Alexa remind me to feed the dog at seven p_m everyday . ",
"Alexa tell me my reminders . ",
"Alexa remember my daughter's birthday is June first . ", 
"Alexa set a timer in five seconds . ",
"Alexa set my alarm for seven a_m tomorrow . ",
"Alexa add oranges and grapes to my shopping list . ",
"Alexa what is in my shopping list ? ",
"Alexa call six zero three six six zero two two zero three . ",
"Alexa hang up . ",
"Alexa find my phone . ",
"Alexa quit . ",
"Alexa turn the bedroom light on . ",
"Alexa turn the bedroom light red . ",
"Alexa change brightness to ten . ",
"Alexa turn off the bedroom light . ",
"Alexa open the kitchen camera . ",
"Alexa hide the kitchen camera . ",
"Alexa play white collar on fire tv . ",
"Alexa pause . ",
"yes . "]


In [50]:
def distance(str1, str2):
    sql = sm(None, str1, str2)
    return sql.ratio()


def categorize(cha_dir, wav_dir, cha_file, wav_file):
    os.chdir(cha_dir)
    cha = ChaFile(cha_file)
    lines = cha.getLines()
    data_set = []
    for data in lines:
        utterance = data["xol"]
        timestamp = data["bullet"]
        utterance = utterance.split(" ")
        dist = []
        edit_dist = []
        for com in COMMANDS:
            dist.append(distance(com, data["xol"]))
            #this method uses the edit distance approach
            utterance = data["xol"]
            edit_dist.append(ed.eval(com, utterance))
        edit_min = min(edit_dist)
        closest_neighbor = COMMANDS[dist.index(max(dist))]
        percent_match = (max(dist))
        data_set.append([utterance, edit_min, percent_match, closest_neighbor])
        frame = pd.DataFrame(data_set)
        frame.columns = ["utterance", "edit min", "percent match", "closest neighbor"]
    # os.chdir("..")
    # with pd.ExcelWriter("editdist.xlsx", mode="a", engine="openpyxl") as writer:
    #     frame.to_excel(writer, sheet_name=cha_file)   
    esum = frame["edit min"].sum()
    psum = frame["percent match"].sum()
    return esum, psum

        


In [51]:
edist_ad = []
pdist_ad = []
for i in range(len(AD_cha_files)):
    sum_edist, sum_pdist = categorize(dir_AD_cha, dir_AD_wav, AD_cha_files[i], AD_wav_files[i])
    edist_ad.append(sum_edist)
    pdist_ad.append(sum_pdist)

edist_mci = []
pdist_mci= []
for i in range(len(MCI_cha_files)):
    sum_edist, sum_pdist = categorize(dir_MCI_cha, dir_MCI_wav, MCI_cha_files[i], MCI_wav_files[i])
    edist_mci.append(sum_edist)
    pdist_mci.append(sum_pdist)

edist_h = []
pdist_h= []
for i in range(len(Healthy_cha_files)):
    sum_edist, sum_pdist = categorize(dir_healthy_cha, dir_healthy_wav, Healthy_cha_files[i], Healthy_wav_files[i])
    edist_h.append(sum_edist)
    pdist_h.append(sum_pdist)

print(edist_ad, pdist_ad)
print(edist_mci, pdist_mci)
print(edist_h, pdist_h)

[96, 32, 290, 108, 33, 40, 66, 11, 54, 431, 95, 263, 156, 92, 228, 9, 164, 28, 47] [35.726423477996704, 34.07191145041998, 47.053117138549624, 38.55662229413788, 35.236802413273, 34.882193557837866, 36.790224518707184, 34.88755622188906, 33.60971338142743, 13.91247266329204, 43.160411551581014, 39.80759107006037, 35.10857947997927, 21.824697754393117, 16.722583648929227, 30.9126213592233, 30.59357704808477, 29.66788787594786, 32.102916356186114]
[12, 36, 88, 504, 206, 44, 106, 80, 123, 45, 85, 19, 19, 82, 149, 31, 131, 80, 460, 91, 27, 29, 0, 130, 76, 88, 70, 19] [37.82322705577607, 36.36915772286855, 34.400717487124105, 47.73859658344965, 37.49003219678676, 36.62887213134921, 41.07391254508235, 40.16885517383767, 47.776267817807344, 38.08710592787394, 39.271887601122685, 38.0119461122884, 40.73704477053123, 42.04833053313534, 42.13910948451125, 40.468404553704005, 47.02824153449588, 37.37580119167873, 40.99335586242703, 41.76207661092429, 37.62550640760645, 37.65015801192272, 38.0, 44

In [52]:
def labeler(classic, term):
    blank = []
    for i in classic:
        blank.append(term)
    return blank


ad_labels = labeler(AD_cha_files, "AD")
mci_labels = labeler(MCI_cha_files,"MCI")
healthy_labels = labeler(Healthy_cha_files, "Healthy")

labels = ad_labels + mci_labels + healthy_labels

def combine(feature1, feature2):
    d = []
    for x in range(len(feature1)):
        s = [feature1[x], feature2[x]]
        d.append(s)
    return d

features = combine(edist_ad, pdist_ad) + combine(edist_mci, pdist_mci) + combine(edist_h, pdist_h)
print(features)

[[96, 35.726423477996704], [32, 34.07191145041998], [290, 47.053117138549624], [108, 38.55662229413788], [33, 35.236802413273], [40, 34.882193557837866], [66, 36.790224518707184], [11, 34.88755622188906], [54, 33.60971338142743], [431, 13.91247266329204], [95, 43.160411551581014], [263, 39.80759107006037], [156, 35.10857947997927], [92, 21.824697754393117], [228, 16.722583648929227], [9, 30.9126213592233], [164, 30.59357704808477], [28, 29.66788787594786], [47, 32.102916356186114], [12, 37.82322705577607], [36, 36.36915772286855], [88, 34.400717487124105], [504, 47.73859658344965], [206, 37.49003219678676], [44, 36.62887213134921], [106, 41.07391254508235], [80, 40.16885517383767], [123, 47.776267817807344], [45, 38.08710592787394], [85, 39.271887601122685], [19, 38.0119461122884], [19, 40.73704477053123], [82, 42.04833053313534], [149, 42.13910948451125], [31, 40.468404553704005], [131, 47.02824153449588], [80, 37.37580119167873], [460, 40.99335586242703], [91, 41.76207661092429], [27

In [125]:
feature_train, feature_test, label_train, label_test = train_test_split(features, labels, test_size=0.33)

In [126]:
clf = svm.SVC()
clf.fit(feature_train, label_train)

SVC()

In [127]:
predict = clf.predict(feature_test)
print(predict)
print(metrics.accuracy_score(label_test, predict))
print(metrics.multilabel_confusion_matrix(label_test, predict))
print(metrics.classification_report(label_test, predict))

['MCI' 'MCI' 'Healthy' 'MCI' 'MCI' 'MCI' 'MCI' 'Healthy' 'MCI' 'Healthy'
 'MCI' 'Healthy' 'MCI' 'AD' 'Healthy' 'Healthy' 'Healthy' 'MCI' 'Healthy'
 'Healthy' 'MCI' 'Healthy' 'Healthy' 'MCI' 'MCI' 'MCI']
0.46153846153846156
[[[20  0]
  [ 5  1]]

 [[10  6]
  [ 5  5]]

 [[ 8  8]
  [ 4  6]]]
              precision    recall  f1-score   support

          AD       1.00      0.17      0.29         6
     Healthy       0.45      0.50      0.48        10
         MCI       0.43      0.60      0.50        10

    accuracy                           0.46        26
   macro avg       0.63      0.42      0.42        26
weighted avg       0.57      0.46      0.44        26



In [128]:
# from tpot import TPOTClassifier
# pipeline_optimizer = TPOTClassifier(max_time_mins=15)
# pipeline_optimizer.fit(np.array(feature_train), np.array(label_train))